In [13]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from scipy.integrate import odeint, solve_ivp
final_fig_path = "../Figures/"
from time import time

In [1]:
%%writefile fun_response_funs.py
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from scipy.integrate import odeint, solve_ivp

    
def fun_response(x,M1,M2,index,a1,a2,h1,h2,**params):
    '''
    functional response to prey as a function of predator group size (x) and 
    prey population sizes (M1, M2)
    a_iα_i(x)M_i/(1 + h_1 a_1 α_1(x) M_1 + h_2 a_2 α_2(x) M_2)
    
    @inputs:
    x - pred group size
    M1 - big prey pop size
    M2 - small prey pop size
    index - 1 (big prey) or 2 (small prey)
    a1 - attack rate of big prey
    a2 - attack rate of small prey
    h1 - handling time per big prey caught
    h2 - handling time per small prey caught
    params: a dictionary of other parameters, that at least must include α1_of_1, α2_of_1, s1, s2

    @returns
    functional response for prey type <index> (a float)

    @examples
    >>fun_response(x=1,M1=10,M2=10,index=1,a1=1,a2=1,h1=0.5,h2=0.5, 
                    **dict(α1_of_1 = 0.05, α2_of_1 = 0.95, s1 = 2, s2 = 2) )
    0.08333333333333336
    (answer should be 10*a1*α1_of_1/(1+h1*a1*α1_of_1*10 + h2*a2*α2_of_1*10) = 0.08333333333333333
    
    '''
    
    α1 = fun_attack_rate(x,1,**params)
    α2 = fun_attack_rate(x,2,**params)
    if index == 1:
        numerator = a1*α1*M1
    elif index == 2:
        numerator = a2*α2*M2
    denominator = 1 + a1*α1*h1*M1 + a2*α2*h2*M2
    return numerator/denominator


def fun_attack_rate(x, index, α1_of_1, α2_of_1, s1, s2, **params):
    '''
    The attack rate as a function of x
    
    @inputs:
    x: group size, 1,2,3,...
    index: 1 or 2, indicates prey type 1 (big prey) or 2 (small prey)
    α1_of_1: the attack rate of big prey for group size 1
    α2_of_1: the attack rate of small prey for group size 1
    s1: critical group size for big prey, must be >= 2
    s2: critical group size for small prey, must be >= 2
    
    @returns:
    attackrate (a float)

    @example:
    >> fun_attack_rate(1,2,0.05,0.95,2,2,**dict())
    0.9500000000000001
    >> fun_attack_rate(1,1,0.05,0.95,2,2,**dict())
    0.05000000000000001
    
    '''
    if index == 1:
        θ_1 = - np.log(1/α1_of_1 - 1)/(1-s1)
        return 1/(1 + np.exp(- θ_1 * (x - s1)))
    elif index == 2:
        θ_2 = - np.log(1/α2_of_1 - 1)/(1-s2)
        return 1/(1 + np.exp(- θ_2 * (x - s2)))
    

Overwriting fun_response_funs.py


In [35]:
%%writefile fitness_funs.py

import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from scipy.integrate import odeint, solve_ivp
from fun_response_funs import *


def fun_fitness(x, M1, M2, **params):
    '''
    this is a subordinate's fitness of being in a group of size x
    @inputs:
    x - grp size
    M1 - pop size of big prey
    M2 - pop size of small prey
    params - a dictionary at least including: b1, b2, r, γ, a1, a2, h1, h2, α1_of_1, 
                                                    α2_of_1, s1, s2
    @returns:
    fitness of a subordinate, a float (or array if one of the parameter inputs is an array)

    @example:
    >>fun_fitness(x=np.array([1,2,3]), M1=10, M2=10, **dict(b1=1,b2=0.1,r=0, γ=0, a1=1, a2=1, h1=0.5, h2=0.5, 
                                                    α1_of_1=0.05, α2_of_1=0.95, s1=2, s2=2))
    array([0.24166667, 0.45833333, 0.53055556])
    '''
    fitnesses = fitness_from_prey(x, M1, M2,**params)
    total_sub_fitness = np.sum(fitnesses,0)
    return total_sub_fitness


def fitness_from_prey(x, M1, M2,b1, b2, r, γ,**params):
    '''
    portion of inclusive fitness from each prey type, stored in an array, after potentially unequal sharing
    @inputs:
    x - pred group size
    M1 - big prey pop size
    M2 - small prey pop size
    b1 - big prey  conversion (prey --> pred)
    b2 - small prey conversion (prey --> pred)
    r - relatedness between group members
    γ - extent of reproductive skew (portion of subordinate's food donated to dominant)
    params - dictionary of other parameters, which must at least contain 
             a1, a2, h1, h2, α1_of_1, α2_of_1, s1, s2

    @returns:
    np.array([<inclusive fitness from big prey>, <inclusive fitness from small prey>])
    (so the rows correspond to prey types

    @example
    >>fitness_from_prey(x= np.array([1,2,3]), M1=10, M2=10, **dict(b1=1,b2=0.1,r=0, γ=0, a1=1, a2=1, h1=0.5, h2=0.5, 
                                                    α1_of_1=0.05, α2_of_1=0.95, s1=2, s2=2))
    array([[0.08333333, 0.41666667, 0.52777778],
       [0.15833333, 0.04166667, 0.00277778]])
       
    '''
    
    wgroup = np.array([b1*fun_response(x,M1,M2,1,**params), b2*fun_response(x,M1,M2,2,**params)])
    try:
        if x > 1:
            repro_exchange = (1-γ)*(1-r) + r*x
            return 1/x *(wgroup) * repro_exchange
        else:
            return wgroup
    except ValueError:
        repro_exchange = np.ones(np.shape(x))
        repro_exchange[x>1] = (1-γ)*(1-r) + r*x[x>1]
        return (1/x) * wgroup * repro_exchange
def fun_fitness_from_big_prey(x, M1, M2, **params):
    '''
    portion of inclusive fitness from big prey type. calls fitness_from_prey
    @inputs:
    x - pred group size
    M1 - big prey pop size
    M2 - small prey pop size
    b1 - big prey  conversion (prey --> pred)
    b2 - small prey conversion (prey --> pred)
    params - dictionary of other parameters, which must at least contain 
             r, γ, a1, a2, h1, h2, α1_of_1, α2_of_1, s1, s2

    @returns:
    <inclusive fitness from big prey>
    '''
    return fitness_from_prey(x, M1, M2, **params)[0]
def fun_fitness_from_small_prey(x, M1, M2, **params):
    '''
    portion of inclusive fitness from small prey type. calls fitness_from_prey
    @inputs:
    x - pred group size
    M1 - big prey pop size
    M2 - small prey pop size
    b1 - big prey  conversion (prey --> pred)
    b2 - small prey conversion (prey --> pred)
    params - dictionary of other parameters, which must at least contain 
             r, γ, a1, a2, h1, h2, α1_of_1, α2_of_1, s1, s2

    @returns:
    <inclusive fitness from small prey>
    '''
    return fitness_from_prey(x, M1, M2, **params)[1]

Overwriting fitness_funs.py


In [ ]:

'''
calls fun_dfdt for the different x's
params is a dictionary of the parameters
'''
def group_formation_model(t, f_of_x_vec,p,M1,M2,params):
    x_max = params["x_max"]
    dfdt = [fun_dfdt(f_of_x_vec, x, p, M1, M2, **params) for x in range(2,x_max+1)]
    #for x in range(2,x_max+1):
    #    dfdt[x-2] = fun_dfdt(f_of_x_vec, x, p, M1, M2, **params)
    return dfdt

'''
fun_dfdt: This calculates the change in the distribution f(x) wrt time for x >= 2


f(x) is the number of groups of size x
f_of_x_vec is a vector of f(x) for x = 2, .., x_max.

τx df/dt = -xf(x)ϕ(x) - f(1) f(x) ψ(x) - f(x) D(x) 
            + f(x+1)ϕ(x+1) + sum_{y=x+1}^{x_max} f(y) D(y)
            
TO-DO: What to do if x = x_max?
'''
def fun_dfdt(f_of_x_vec, x, p, M1, M2,  τx, δ, x_max, **params):

    # get f(x), f(1), and f(x+1)
    
    def f(y):
        if y == 1:
            return p - sum([z*f(z) for z in range(2,x_max+1)]) # this is recursively designed
        if y >= 2 and y <= x_max:
            return f_of_x_vec[y-2]
        else:
            return 0
        #return(fun_f_of_x(y,x_max,**params))

    def D_tot(y):
        return fun_1_death(y, τx, δ, **params)
    def D(z1,z2):
        # probability group of size z2 shrinks to group of size z1 for z1 \leq x_max  
        return fun_death_y_to_x(z1,z2, τx, δ, x_max, **params) if z2 <= x_max else 0
    def ϕ(y):
        # probability individual leaves group of size y for y <= x_max
        return fun_leave_group(y, M1, M2, x_max, **params) if y<= x_max else 0
    def ψ(y):
        return fun_join_group(y, M1, M2, x_max, **params) if y < x_max else 0
    
        
    individual_leaves = x*f(x) * ϕ(x)
    grows_to_larger_group = f(1)*f(x) * ψ(x)
    death_in_group = f(x) * D_tot(x)
    
    if x == 2:
        #join_smaller_grp = f(1)*f(x-1)*ψ(x-1)
        join_smaller_grp = (1/2)*f(1)*(f(1)-1)*ψ(1)
    else:
        join_smaller_grp = f(1)*f(x-1)*ψ(x-1)
            
    #join_smaller_grp = f(1)*f(x-1)*ψ(x-1)
    larger_grp_shrinks = (x+1)*f(x+1)*ϕ(x+1)
    death_in_larger_grp = sum([f(y)*D(x,y) for y in range(x+1,x_max+1)])
        
    #dfdt_times_taux = x*f_of_x*fun_leave_group(x) - 
    
    
    return 1/τx * (-individual_leaves - grows_to_larger_group - death_in_group  
                  + join_smaller_grp + larger_grp_shrinks + death_in_larger_grp)

'''

'''
def fun_f_of_x(y, f_of_x_vec, p, x_max,**params):
    if y == 1:
            return p - sum([z*f_of_x_vec[z-2] \
                            for z in range(2,x_max+1)]) 
    if y >= 2 and y <= x_max:
        return f_of_x_vec[y-2]
    else:
        return 0
    
'''
The probability an individual leaves a group of size x.
This is ϕ(x) in the text
FILL IN
'''
def fun_leave_group(x, M1, M2, x_max, **params):
    return best_response_fun(1,x,M1,M2,**params)

'''
The probability an individual joins a group of size x.
This is ψ(x) in the text
FILL IN
'''
def fun_join_group(x, M1, M2, x_max, **params):
    # deciding between being alone or being in a group of size x + 1
    return best_response_fun(x+1,1,M1,M2,**params)
'''
Compares W(x) to W(y) to "decide" on group size y or x
'''
def best_response_fun(x,y,M1,M2, d, **params):
    W_of_x = fun_fitness(x,M1,M2, **params)
    W_of_y = fun_fitness(y, M1, M2, **params)
    return W_of_x**d/(W_of_x**d + W_of_y**d)

def nchoosek(n,k):
    return sp.special.factorial(n)/(sp.special.factorial(k)*sp.special.factorial(n-k))

'''
The probability a group of size y shrinks to a group of size x because y - x individuals die, 
# works for for x < y, y <= x_max
'''
def fun_death_y_to_x(x, y, τx, δ, x_max, **params):
    
    return nchoosek(y,y-x) * (δ*τx)**(y-x)*(1-δ*τx)**x

def fun_1_death(x, τx, δ, **params):
    return 1 - (1 - δ*τx)**x


    
    

def fun_num_groups(f_of_x_vec,p,x_max):
    total = 0
    for x in range(1,x_max+1):
        total += fun_f_of_x(x, f_of_x_vec, p, x_max,**dict())
    return total

'''
checks that dfdt = 0
needs to be checked
'''
def check_at_equilibrium(f_of_x_vec, p, M1, M2,  x_max, **params):
    for x in range(2, x_max+1):
        dfdt = fun_dfdt(f_of_x_vec, x, p, M1, M2, x_max=x_max,**params)
        at_equilibrium = np.abs(dfdt)<1e-5
        if not at_equilibrium:
            return 0
    return 1